In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
from IPython.display import Audio
from scipy.io.wavfile import write
from datetime import datetime
import subprocess
from scipy.interpolate import LinearNDInterpolator

In [ ]:
def get_interpolated(array, index):
    if not hasattr(array, "__len__"): return array # if scalar
    return (1 - (index[0] % 1)) * get_interpolated(array[int(np.floor(index[0]))], index[1:]) + (index[0] % 1) * get_interpolated(array[int(np.ceil(index[0]))], index[1:])

In [ ]:
def vectorized_interpolation(array, indices): # indices.shape (dim_array, desired number of interpolations)
    if len(array.shape) == 0: return array
    t = indices[0] % 1
    floor_indices = np.floor(indices[0]).astype(int)
    ceil_indices  = np.ceil (indices[0]).astype(int)
    return (1 - t) * vectorized_interpolation(array[floor_indices], indices[1:]) + t * vectorized_interpolation(array[ceil_indices], indices[1:])

In [ ]:
vectorized_interpolation(np.array([[0, 1, 2], [10, 20, 30]]), np.array([0.5, 0.5]))

In [ ]:
def calculate_next_psi(psi, dt, potential):
    n = psi.shape[0]
    next_psi = np.zeros((n, n), dtype=np.complex_)

    # potential-part
    next_psi = psi * np.exp(1j * dt * potential)

    next_psi = np.fft.fft2(next_psi)

    indices = 2 * np.pi * np.min([np.arange(n), n-np.arange(n)], axis=0)
    k = indices.reshape(-1, 1)
    l = indices.reshape(1, -1)
    theta = (k*k + l*l) * dt
    next_psi *= np.exp(1j * theta)

    next_psi = np.fft.ifft2(next_psi)
    return next_psi

In [ ]:
def gaussian(x, y, n, offset, width):
    x = (x - n/2.0) / (n/2.0) - offset[0]
    y = (y - n/2.0) / (n/2.0) - offset[1]
    return np.exp(-(x*x + y*y) / (width*width)) + 0j

In [ ]:
def parabolar(x, y, n, offset, factor):
    x = (x - n/2.0) / (n/2.0) - offset[0]
    y = (y - n/2.0) / (n/2.0) - offset[1]
    return factor * (x*x + y*y)
    

In [ ]:
parabolar(64, 0, 128, [0, 0], 1)

In [1]:
def circle(rad, radius, offset, n):
    return (radius * np.array([np.cos(rad), np.sin(rad)]) + offset) * n//2 + n//2

In [ ]:
np.concatenate(([5], circle(0, 1, 0, 128)), axis=0)

In [ ]:
n = 128

potential = np.array([[parabolar(x, y, n, offset=[0,0], factor=10000) for x in range(n)] for y in range(n)])

# barrier
double_slit = [(-4, -2), (2, 4)]
single_slit = [(-2, 2)]
slits = double_slit

barrier_height = 1e60
barrier = [barrier_height] * n
for s in slits:
    barrier[n//2+s[0]:n//2+s[1]] = [0] * (s[1]-s[0])
for i in range(n):
    potential[:, n//2-1] += barrier

In [ ]:
sim_fps = 400
duration = 5
frame_amount = duration * sim_fps
simulation_speed = 0.004

frames = np.empty((frame_amount, n, n), dtype=complex) # for storing the generated images

psi = np.array([[gaussian(x, y, n, offset=[-0.6, 0.0], width=0.15) for x in range(n)] for y in range(n)])
frames[0] = psi

#plt.pcolormesh(pow(np.abs(frames[0]), 2.0/3.0), cmap='inferno', vmin=0, vmax=1)
plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()


for i in range(1, frame_amount):
    psi = calculate_next_psi(psi, simulation_speed / sim_fps, potential)
    frames[i] = psi

print("Finished simulation")

# Video

In [ ]:
# create visual barrier for plot
def visual_barrier(barrier_gaps):
    start = 0
    rects = []
    for g in barrier_gaps:
        end = n//2 + g[0]
        rect = patches.Rectangle((n//2 - 1.5, start - 0.5), 1, end - start, linewidth=0, facecolor='#60b0ff')
        start = n//2 + g[1]
        rects.append(rect)
    rects.append(patches.Rectangle((n//2 - 1.5, start - 0.5), 1, n - start, linewidth=0, facecolor='#60b0ff'))
    return rects

In [ ]:
# FuncAnimation
video_fps = 20

fig, ax = plt.subplots()
plt.axis('off')  # big performance boost

data = pow(np.abs(frames[0]), 2.0/3.0)
cax = ax.imshow(data, cmap='inferno', vmin=0, vmax=1)
for b in visual_barrier(slits):
    ax.add_patch(b)
fig.colorbar(cax)  # no performance impact (?)

def animate(i):
    cax.set_array(pow(np.abs(frames[i * sim_fps // video_fps]), 2.0/3.0))

anim = animation.FuncAnimation(fig, animate, frames=frame_amount * video_fps // sim_fps)
video_filename = f'output/simulation_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.mp4'

anim.save(video_filename, fps=video_fps, dpi=150, bitrate=4000)

print(f'video saved as {video_filename}')

# Sonification

In [ ]:
sample_rate = 44100
frequency = 220
radius = 0.6

In [ ]:
# Sonification
frames_indices = np.arange(duration * sample_rate) / sample_rate * sim_fps
radians_per_sample = 2 * np.pi * frequency / sample_rate
radians = np.arange(duration * sample_rate) * radians_per_sample
x = np.cos(radians)
y = np.sin(radians)
interpolator = LinearNDInterpolator(np.array([frames_indices.ravel(), x.ravel(), y.ravel()]).T, frames.ravel())
audio = np.square(np.abs(interpolator((frames_indices, x, y), frames)))

print("Finished sonification")

In [ ]:
audio[:1000]  *= np.square(np.linspace(start=0, stop=1, num=1000, endpoint=False))
audio[-1000:] *= np.square(np.linspace(start=1, stop=0, num=1000, endpoint=False))
audio_filename = f'sonification_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.wav'
write(audio_filename, sample_rate, np.round((audio - np.average(audio)) / np.max(np.abs(audio - np.average(audio))) * 32767).astype(np.int16))
print(f"Sonification saved as {audio_filename}")
Audio(audio, rate=sample_rate)

In [ ]:
plt.plot(audio)

# Combine Video & Audio

In [ ]:
combined_filename = f'combination_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.mp4'

# Construct the ffmpeg command to combine video and audio
ffmpeg_command = [
    'ffmpeg',
    '-i', video_filename,   # Input video file
    '-i', audio_filename,   # Input audio file
    '-c:v', 'copy',         # Copy the video stream
    '-c:a', 'aac',          # Encode the audio to AAC (necessary for some formats)
    '-shortest',            # Finish encoding when the shortest input stream ends
    combined_filename         # Output file
]

# Execute the command
subprocess.run(ffmpeg_command)